# Load and handle data from a CSV file

# Create a dataframe which keeps track of the combinations

In [32]:
import pandas as pd
import itertools
import numpy as np

'''When loading data, it is extremely important to have every descriptor column filled for each reagent.
If not, certain rows will get dropped when the .dropna() function is called in the updated dataframe.'''

reagent_data = pd.read_csv('/Users/alanwortman/Desktop/Well Plate Example 2.csv')
reagent_data

'''This function takes a CSV file of what's on the well plate and adds a 
column with the coordinates for each well as a tuple. 
CSV files must use the template for this to work.'''

def create_coordinates(dataframe): 
    coordinates = []
    for i in reagent_data.index:
        coordinates.append((reagent_data.x[i], reagent_data.y[i]))
    dataframe['coordinates'] = coordinates    

#Calling the function to the dataframe specified above ^^^
#Creates a new dataframe with the added coordinates column
create_coordinates(reagent_data)
reagent_data_updated = reagent_data.dropna().reset_index(drop = True)

#Create lists of all the columns (as strings), rows (as strings), and coordiantes (as tuples)
columns = list(map(str,reagent_data_updated.plate_column.tolist())) # must be a string to concatenate with rows
rows = reagent_data_updated.plate_row.tolist()
coordinates = reagent_data_updated.coordinates.tolist()

wells = [r + c for r,c in zip(rows, columns)] #what does this do? (it is a list of all the wells)
wells_coords = [(r + c, (coord)) for r,c, coord in zip(rows, columns, coordinates)]

#Making the full factorial well combinations as a list
'''Here, components_coords is a list of lists, where each inner list contains the well coordinates 
for a particular component. The * operator is used to unpack this list of lists into separate arguments 
for the product function.'''
components = sorted(list(set(rows))) #list of all the individual types of rows (ex. A, B, C)
components_coords = [[w for w in wells_coords if w[0][0] == c] for c in components] #list of all the wells and their corresponding coordinates
combinations = list(itertools.product(*components_coords)) #list of all the combinations


# Create a pandas dataframe to store the combinations. 
# This will make it easy to track which reagents were in each droplet

number_wells_to_sample = range(len(set(rows))) #calculates the number of different rows to go to for all the combinations

#Takes all the combinations and creates a dataframe from it. Calculates the number of columns and names them based on the number of rows
#Each row will be a different variable to test (ex. photocatalyst, base, etc)
df_combinations = pd.DataFrame(combinations, columns = list('well_'+ str(i+1) for i in number_wells_to_sample))

#Goes through the dataframe, makes a list of all the coordinates for a combo, and adds them to a new column
df_combinations['coordinates'] = df_combinations.apply(lambda x: tuple([val[1] for val in x]), axis=1)

#Drops all the coordinates associated with a well to simplify the dataframe
df_combinations.iloc[:, :-1] = df_combinations.iloc[:, :-1].applymap(lambda x: x[0])
df_combinations.head()


,well_1,well_2,well_3,coordinates
0,A1,B1,C1,"((0.0, 0.0), (0.0, 4.5), (0.0, 9.0))"
1,A1,B1,C2,"((0.0, 0.0), (0.0, 4.5), (4.5, 9.0))"
2,A1,B2,C1,"((0.0, 0.0), (4.5, 4.5), (0.0, 9.0))"
3,A1,B2,C2,"((0.0, 0.0), (4.5, 4.5), (4.5, 9.0))"
4,A1,B3,C1,"((0.0, 0.0), (9.0, 4.5), (0.0, 9.0))"


# Write Gcode from the combinations

### Notes

In [9]:
# combinations = all the combinations
# combinations[i] = specific combination
# combinations[i][j] = well
# combinations[i][j][k] = coordinates for k = 1
# combinations[i][j][k][l] = x if l = 0, y if l = 1

In [38]:
'''When we add code for the pump stop/start, we will want to specify parameters for how long those pauses are. 
Ideally this will be passed through a function where a keyword argument specifies the length of those pauses. 
Additionally, we will want to do this for the pauses of the CNC in the wells'''

'When we add code for the pump stop/start, we will want to specify parameters for how long those pauses are. \nIdeally this will be passed through a function where a keyword argument specifies the length of those pauses. \nAdditionally, we will want to do this for the pauses of the CNC in the wells'

In [36]:
# !!!In this case, the PFD well is defined as combinations[i][0] or the first well that's always sampled!!!

#This function is set up to take keyword arguments
code_lines = []
def write_Gcode(*, combinations, PFD_dwell, organic_dwell, pause_after_pump_stop):
    code_lines.append(["M8 M9"])
    for i in range(len(combinations)):
        for k in range(len(combinations[1])):
            if k == 0:
                code_lines.append([f'G01 X{combinations[i][k][1][0]} Y{combinations[i][k][1][1]} Z0.0 F3000'])
                code_lines.append([f'G01 X{combinations[i][k][1][0]} Y{combinations[i][k][1][1]} Z-5.5 F3000'])
                code_lines.append(["M3 S0"]) #pump start
                code_lines.append(["M8 M9"])
                code_lines.append([f'G04 P{PFD_dwell}']) #CNC dwell (longer for PFD)
                code_lines.append(["M3 S500"]) #pump pause
                code_lines.append(["M8 M9"])
                code_lines.append([f'G04 P{pause_after_pump_stop}']) #CNC pause for pump to catch up
                code_lines.append([f'G01 X{combinations[i][k][1][0]} Y{combinations[i][k][1][1]} Z0.0 F3000'])
                code_lines.append([''])
            else:
                code_lines.append([f'G01 X{combinations[i][k][1][0]} Y{combinations[i][k][1][1]} Z0.0 F3000'])
                code_lines.append([f'G01 X{combinations[i][k][1][0]} Y{combinations[i][k][1][1]} Z-5.5 F3000'])
                code_lines.append(["M3 S0"]) #pump start
                code_lines.append(["M8 M9"]) 
                code_lines.append([f'G04 P{organic_dwell}']) #CNC dwell (shorter for organic)
                code_lines.append(["M3 S500"]) #pump pause
                code_lines.append(["M8 M9"])
                code_lines.append([f'G04 P{pause_after_pump_stop}']) #CNC pause for pump to catch up
                code_lines.append([f'G01 X{combinations[i][k][1][0]} Y{combinations[i][k][1][1]} Z0.0 F3000'])
                code_lines.append([''])
                

#example of calling the function:              
#write_Gcode(combinations = combinations, PFD_dwell = 2, organic_dwell = 1, pause_after_pump_stop = 2)
            
            
'''Summary of codes:
G00 fast move
G01 linear interpolation
G04 dwell

F3000 feedrate command

Px pause for x amount of time during the dwell

M08 M09 required as the first line of every code for the pump/valve to actuate properly (Doug Dickey)
        these are also required after any of the M/S commands (Doug Dickey)
        
M3 S0 start pump (Doug Dickey)
M3 S500 pause pump (Doug Dickey)
M3 S1000 stop pump (Doug Dickey)

M4 S0 position A on valve (Doug Dickey)
M4 S1000 position B on valve (Doug Dickey)'''

'Summary of codes:\nG00 fast move\nG01 linear interpolation\nG04 dwell\n\nF3000 feedrate command\n\nPx pause for x amount of time during the dwell\n\nM08 M09 required as the first line of every code for the pump/valve to actuate properly (Doug Dickey)\n        these are also required after any of the M/S commands (Doug Dickey)\n        \nM3 S0 start pump (Doug Dickey)\nM3 S500 pause pump (Doug Dickey)\nM3 S1000 stop pump (Doug Dickey)\n\nM4 S0 position A on valve (Doug Dickey)\nM4 S1000 position B on valve (Doug Dickey)'

In [30]:
np.savetxt("Example_GCodes.txt", code_lines, fmt = '%s')

In [34]:
# write_Gcode(combinations = combinations, PFD_dwell = 2, organic_dwell = 1, pause_after_pump_stop = 2)
# code_lines